# Team 6 Data Vizualization Project - County Research Data 
#### Goal is to collect the value of various parameters to research on counties in the United States. The parameters we have considered for data for population, immigration, crime, education, unemplyment, poverty, median house hold income, education and property rates.

In [4]:
import pandas as pd

### FIPS Data - FIPS Code, State Name, County Name
<p> Using FIPS code as a standard reference across all the files </p>

In [5]:
FIPS_data=pd.read_excel('US_FIPS_Codes.xls',skiprows=1, converters={'FIPS County':str, 'FIPS State': str})
FIPS_data["FIPS"] = FIPS_data["FIPS State"].map(str) + FIPS_data["FIPS County"]
FIPS_data = FIPS_data[['State', 'County Name', 'FIPS']]

In [6]:
FIPS_data.head(5)

,State,County Name,FIPS
0,Alabama,Autauga,01001
1,Alabama,Baldwin,01003
2,Alabama,Barbour,01005
3,Alabama,Bibb,01007
4,Alabama,Blount,01009


### FIPS code is represented in mutiple ways across the files. Below are 2 functions to have a consistent 5-digit single FIPS code in for the counties. 

In [7]:
# Function to convert 4 digit FIPS code to 5 digit FIPS code

def buffer_FIPS(row):
    if len(str(row['FIPS']))==4:
        return "0"+str(row['FIPS'])
    else:
        return str(row['FIPS'])

In [8]:
# Function to convert 2 and 1 digit FIPS County codes to 3 digit code

def buffer_FIPS_CTY(row):
    if len(str(row['FIPS_CTY']))==2:
        return "0"+str(row['FIPS_CTY'])
    elif len(str(row['FIPS_CTY']))==1:
        return "00"+str(row['FIPS_CTY'])
    else:
        return str(row['FIPS_CTY'])
    

### Population Estimate Data

In [9]:
population_data = pd.read_excel('PopulationEstimates.xls', skiprows=2)

# Retaining required rows
population_data = population_data[['FIPS','POP_ESTIMATE_2015']]

# Having a consistant 5 digit FIPS code
population_data['FIPS']=population_data.apply(buffer_FIPS,axis=1)

In [10]:
population_data.head (5)

,FIPS,POP_ESTIMATE_2015
0,0,320896618
1,01000,4853875
2,01001,55035
3,01003,203690
4,01005,26270


### Immigration Data
This dataset does not have FIPS code instead has County name and State name. Each county name contains 'County' as in 'Autauga County'. Below is the procedure followed to get the FIPS code from FIPS_data file
* Create a new column in immigration_data called 'county&state' such that it contains County name (strip the string 'County' from County name and concatenate it with State name 
* Create a new column in FIPS_data called 'county&state' with same rule as above
* Join the 2 datasets and retain only the required columns

PS : Did a search on FIPS_Data - County Name column to check if any of the county's have the string 'County' in them. No county has the string 'County' in its name (including Orange County)

In [11]:
immigration_data = pd.read_csv('Immigration_County _wise.csv')
immigration_data = immigration_data [['County','State','County Immigrants']]
immigration_data['county&state'] = immigration_data['County'].map(lambda x: x.rstrip('County')).map(str) + immigration_data['State']
FIPS_data['county&state'] = FIPS_data['County Name'].map(str) +' '+ FIPS_data['State']
immigration_data = pd.merge(immigration_data, FIPS_data, on='county&state', how='outer')
immigration_data = immigration_data[['FIPS','County Immigrants']]

In [12]:
immigration_data.head(5)

,FIPS,County Immigrants
0,01001,861.0
1,01003,6882.0
2,01005,737.0
3,01007,251.0
4,01009,2393.0


### Unemplyment and Median Houseld Income data

In [13]:
unemp_income_data = pd.read_excel('Unemployment.xls', skiprows = 7,converters={'FIPStxt':str})

unemp_income_data = unemp_income_data.rename(columns={'FIPStxt': 'FIPS'})

# Retaining only the required columns
unemp_income_data = unemp_income_data[['FIPS','Unemployment_rate_2015','Median_Household_Income_2015']]

In [14]:
unemp_income_data.head(5)

,FIPS,Unemployment_rate_2015,Median_Household_Income_2015
0,01000,6.1,44833
1,01001,5.2,56580
2,01003,5.5,52387
3,01005,8.9,31433
4,01007,6.6,40767


### Education Data

In [15]:
education_data = pd.read_excel('Education.xls', skiprows=4, converters={'FIPS Code':str})

# Renaming the columns
education_data = education_data.rename(columns={"FIPS Code": "FIPS",
                "Bachelor's degree or higher, 2011-2015" : "Bachelor's degree/higher(2015)-Absolute",
                "Percent of adults with a bachelor's degree or higher, 2011-2015" : "Bachelor's degree/higher(2015)-Percentage"})

# Retaining only required fields
education_data = education_data[["FIPS","Bachelor's degree/higher(2015)-Absolute","Bachelor's degree/higher(2015)-Percentage"]]

In [45]:
education_data.head(5)

,FIPS,Bachelor's degree/higher(2015)-Absolute,Bachelor's degree/higher(2015)-Percentage
0,00000,62952272.0,29.8
1,01000,761650.0,23.5
2,01001,8437.0,23.2
3,01003,39710.0,29.0
4,01005,2354.0,12.5


### Poverty Estimate Data

In [46]:
poverty_data = pd.read_excel('PovertyEstimates.xls', skiprows=3, converters={'FIPStxt':str})

poverty_data = poverty_data.rename(columns={'FIPStxt': 'FIPS', 'POVALL_2015': 'Povert_Estimate(2015)'})

# Retaining only required columns 
poverty_data = poverty_data[['FIPS','Povert_Estimate(2015)']]

In [47]:
poverty_data.head(5)

,FIPS,Povert_Estimate(2015)
0,00000,46153077
1,01000,875853
2,01001,6966
3,01003,25941
4,01005,7380


### Crime Data

In [48]:
crime_data = pd.read_csv('crime_data_w_population_and_crime_rate.csv',converters={'FIPS_ST':str, 'FIPS_CTY':str})

# Converting FIPS_CTY codes to 3 digit codes
crime_data['FIPS_CTY'] = crime_data.apply(buffer_FIPS_CTY,axis=1)

# Concatenating FIPS_CTY and FIPS_State
crime_data['FIPS'] = crime_data['FIPS_ST'].map(str) + crime_data['FIPS_CTY']

# Converting FIPS codes to 5 digit codes
crime_data['FIPS'] = crime_data.apply(buffer_FIPS,axis=1)

# Retaining only required columns
crime_data = crime_data[['FIPS','crime_rate_per_100000']]

In [49]:
crime_data.head(5)

,FIPS,crime_rate_per_100000
0,29510,1791.995377
1,05035,1754.914968
2,17003,1664.700485
3,48261,1456.310680
4,22031,1447.402430


### Property Rates Data (per square feet)

In [50]:
property_rate_data = pd.read_csv('County_MedianListingPricePerSqft_AllHomes.csv',
                     converters={'StateCodeFIPS':str, 'MunicipalCodeFIPS':str})

# Creating a FIPS code column to have the State FIPS and Municipal Codes(countys) to a 5 digit
property_rate_data['FIPS'] = property_rate_data['StateCodeFIPS'].map(str) + property_rate_data['MunicipalCodeFIPS']

# Renaming the property rate field
property_rate_data = property_rate_data.rename(columns={'2015-12': 'Cost/FtSq (2015-Dec)'})

# Retaining required columns
property_rate_data = property_rate_data[['FIPS', 'Cost/FtSq (2015-Dec)']]

In [51]:
property_rate_data.head(5)

,FIPS,Cost/FtSq (2015-Dec)
0,06037,358.802817
1,17031,152.945097
2,48201,110.309699
3,04013,139.335726
4,06073,318.531062


### Merging all the files on FIPS column
FIPS_data contains all the rows at County level.
Several other datasets contain county level data along with data aggregated at state level. So initial merge (FIPS_Data and Population_Data) is on left join so as to retain only the columns which contain county data. 

In [52]:
FIPS_data = FIPS_data[['State','County Name','FIPS']]

county_research_data = FIPS_data.merge(population_data,on='FIPS', how = 'left')\
                .merge(immigration_data,on='FIPS')\
                .merge(unemp_income_data,on='FIPS')\
                .merge(education_data,on='FIPS')\
                .merge(poverty_data,on='FIPS')\
                .merge(crime_data,on='FIPS')\
                .merge(property_rate_data,on='FIPS')
        

In [53]:
county_research_data.head(5)

,State,County Name,FIPS,POP_ESTIMATE_2015,County Immigrants,Unemployment_rate_2015,Median_Household_Income_2015,Bachelor's degree/higher(2015)-Absolute,Bachelor's degree/higher(2015)-Percentage,Povert_Estimate(2015),crime_rate_per_100000,Cost/FtSq (2015-Dec)
0,Alabama,Autauga,01001,55035.0,861.0,5.2,56580,8437.0,23.2,6966,251.601926,89.477541
1,Alabama,Baldwin,01003,203690.0,6882.0,5.5,52387,39710.0,29.0,25941,228.086325,132.849394
2,Alabama,Barbour,01005,26270.0,737.0,8.9,31433,2354.0,12.5,7380,177.278771,87.353059
3,Alabama,Bibb,01007,22561.0,251.0,6.6,40767,1664.0,10.6,4516,217.661692,87.437500
4,Alabama,Blount,01009,57676.0,2393.0,5.4,50487,5080.0,12.9,8399,210.810064,90.436346


In [54]:
# Writing the combinedbdataframe into excel file with no index
county_research_data.to_csv('County_Research_Data.csv', index=False)